## Dependencies

In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
from evaluate import load
import torch
import pandas as pd
from tqdm import tqdm

[2025-01-25 16:26:36,946: WARNING: module_wrapper: From c:\Projects\python\mainenv\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2025-01-25 16:26:37,424: INFO: config: PyTorch version 2.5.1+cu118 available.]
[2025-01-25 16:26:37,427: INFO: config: TensorFlow version 2.18.0 available.]


In [2]:
%pwd

'c:\\Projects\\python\\text-summarizer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Projects\\python\\text-summarizer'

## Entity

In [5]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file: Path

## Configuration manager

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        # paths accessed via constant.py -> config.yaml,params.yaml
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root]) # due to configbox setup

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file=config.metric_file
        )
        return model_evaluation_config

## Components

In [7]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self,list_of_elements,batch_size):
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]
    
    def calculate_metric_on_test_ds(
            self,
            dataset,
            metric,
            model,
            tokenizer,
            batch_size=16,
            device = "cuda" if torch.cuda.is_available() else "cpu",
            column_text="article",
            column_summary="highlights"):
        
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text],batch_size=batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size=batch_size))

        for article_batch, target_batch in tqdm(zip(article_batches,target_batches),total = len(article_batches)):

            inputs = tokenizer(article_batch,
                               max_length=1024,
                               truncation=True,
                               padding="max_length",
                               return_tensors="pt")
            summaries = model.generate(
                input_ids = inputs['input_ids'].to(device),
                attention_mask=inputs['attention_mask'].to(device),
                length_penalty = 0.8,
                num_beams=8,
                max_length=128)
            
            # Decode the summaries
            decoded_summaries = tokenizer.batch_decode(summaries, skip_special_tokens=True)

            # Preprocess summaries and references
            decoded_summaries = [d.replace(""," ") for d in decoded_summaries]
            # target_batch = [target.strip() for target in target_batch]      
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

  
        rouge_metric = load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset=dataset_samsum_pt['test'][0:10],
            metric=rouge_metric,
            model=model,
            tokenizer=tokenizer,
            batch_size= 2,
            column_text = 'dialogue', 
            column_summary= 'summary'
            )

        df = pd.DataFrame(score, index = ['distalBART'])
        df.to_csv(self.config.metric_file, index=False)

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    print(f"Error occurred: {e}")
    raise e

[2025-01-25 16:26:37,755: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-25 16:26:37,757: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-25 16:26:37,757: INFO: common: created directory at: artifacts]
[2025-01-25 16:26:37,758: INFO: common: created directory at: artifacts/model_evaluation]


c:\Projects\python\mainenv\Lib\site-packages\transformers\models\bart\configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
  0%|          | 0/5 [00:00<?, ?it/s]c:\Projects\python\mainenv\Lib\site-packages\transformers\generation\utils.py:1528: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
100%|██████████| 5/5 [00:12<00:00,  2.41s/it]

[2025-01-25 16:26:53,074: INFO: rouge_scorer: Using default tokenizer.]
